In [1]:
import numpy as np
import pandas as pd
import os
import keras

2024-11-08 15:47:03.327883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 15:47:03.784416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 15:47:03.953503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 15:47:05.079026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
current_path = os.getcwd()
os.chdir(current_path)
print(f"Working directory set to: {current_path}")

Working directory set to: /sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project


In [3]:
import pandas as pd
import glob

# Get all CSV file paths
csv_files = glob.glob("/sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project/final_data/*.csv")

csv_files_sorted = sorted(csv_files)

# Read and concatenate all CSV files into a single DataFrame
dataframes = [pd.read_csv(file) for file in csv_files_sorted]
combined_df = pd.concat(dataframes, ignore_index=True)

# Inspect the combined DataFrame
#combined_df.head()

In [4]:
X_data = combined_df[['insurance','race','marital_status','gender','anchor_age','value','amount','amountuom','label','sequence_num']]

In [5]:
data_encoded = pd.get_dummies(X_data, columns=['insurance','race','marital_status','gender','amountuom','label','value'])

In [6]:
#NaN values in amount
data_encoded.isna().sum()

anchor_age                      0
amount                      15856
sequence_num                    0
insurance_Medicaid              0
insurance_Medicare              0
                            ...  
value_SIMV/PSV/AutoFlow         0
value_SPONT                     0
value_Sensor Medic (HFO)        0
value_Standby                   0
value_VOL/AC                    0
Length: 104, dtype: int64

In [7]:
 data_encoded['amount'] = data_encoded['amount'].fillna(0)

In [8]:
data_encoded.isna().sum().sum()

0

In [9]:
from sklearn.model_selection import train_test_split

# Split into features (X) and target labels (y)
X = data_encoded.values  
X = X.astype(np.float32)
y = combined_df['Died'].values  
y = y.astype(np.int32)

# Split data into training, validation, and testing sets (e.g., 70% train, 15% val, 15% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

Training set size: 850479
Validation set size: 182245
Testing set size: 182246


In [10]:
import tensorflow as tf

# Define a function to convert arrays to TensorFlow Datasets
def to_tf_dataset(X, y, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(X)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

#def to_tf_dataset(X, y, batch_size=32):
#    dataset = tf.data.Dataset.from_tensor_slices((X, y))
 #   dataset = dataset.shuffle(buffer_size=len(X)).padded_batch(
#        batch_size,
#        padded_shapes=([104,], []),
#        padding_values=(0.0, 0)
#        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#    return dataset

# Create TensorFlow Datasets
train_dataset = to_tf_dataset(X_train, y_train)
val_dataset = to_tf_dataset(X_val, y_val)
test_dataset = to_tf_dataset(X_test, y_test)


2024-11-08 15:47:35.432815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46853 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:61:00.0, compute capability: 8.6


In [11]:
# Inspect the dataset
for x, y in train_dataset.take(1):
    print(x, y)

tf.Tensor(
[[ 70.  74.  99. ...   0.   0.   0.]
 [ 67. 124. 223. ...   0.   0.   0.]
 [ 79.  80.  78. ...   0.   0.   0.]
 ...
 [ 77.  88. 451. ...   0.   0.   0.]
 [ 87.  11. 351. ...   0.   0.   0.]
 [ 67.  80.  55. ...   0.   0.   0.]], shape=(32, 104), dtype=float32) tf.Tensor([0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0], shape=(32,), dtype=int32)


2024-11-08 15:47:41.791040: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
element_spec = train_dataset.element_spec
print(element_spec)
#train_dataset.tf.shape

(TensorSpec(shape=(None, 104), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [13]:
#model 1
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1,input_shape = (None,1))
])

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer,
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)]
             )
history = model.fit(train_dataset, epochs=10,
                    validation_data=(val_dataset))

/home/rmd9ev/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


I0000 00:00:1731098886.191180  875957 service.cc:146] XLA service 0x56304da2a560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731098886.191233  875957 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-11-08 15:48:06.220152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-08 15:48:06.341418: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8904


   43/26578 ━━━━━━━━━━━━━━━━━━━━ 1:44 4ms/step - binary_accuracy: 0.7550 - loss: 2.1356

I0000 00:00:1731098886.965833  875957 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26578/26578 ━━━━━━━━━━━━━━━━━━━━ 110s 4ms/step - binary_accuracy: 0.7512 - loss: 1.8763 - val_binary_accuracy: 0.7510 - val_loss: 0.5616
Epoch 2/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7507 - loss: 0.5617 - val_binary_accuracy: 0.7510 - val_loss: 0.5613
Epoch 3/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7506 - loss: 0.5618 - val_binary_accuracy: 0.7510 - val_loss: 0.5615
Epoch 4/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7506 - loss: 0.5617 - val_binary_accuracy: 0.7510 - val_loss: 0.5612
Epoch 5/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7509 - loss: 0.5614 - val_binary_accuracy: 0.7510 - val_loss: 0.5615
Epoch 6/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7510 - loss: 0.5613 - val_binary_accuracy: 0.7510 - val_loss: 0.5612
Epoch 7/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7512 - loss: 0.5611 - val_binary_accuracy: 0

In [14]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - binary_accuracy: 0.7530 - loss: 0.5593
Loss:  0.5597178339958191
Accuracy:  0.7525377869606018


In [16]:
# model 2
# Build the model
model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0.0, input_shape=(None, 1)),  # for variable-length sequences
    keras.layers.SimpleRNN(32),  # SimpleRNN with 32 units
    keras.layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_dataset, epochs=5,
                    validation_data=(val_dataset))

Epoch 1/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 266s 10ms/step - accuracy: 0.7691 - loss: 0.4878 - val_accuracy: 0.7860 - val_loss: 0.4799
Epoch 2/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 265s 10ms/step - accuracy: 0.7742 - loss: 0.4825 - val_accuracy: 0.7846 - val_loss: 0.4761
Epoch 3/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 266s 10ms/step - accuracy: 0.7754 - loss: 0.4819 - val_accuracy: 0.7830 - val_loss: 0.4791
Epoch 4/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 269s 10ms/step - accuracy: 0.7742 - loss: 0.4816 - val_accuracy: 0.7570 - val_loss: 0.4921
Epoch 5/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 267s 10ms/step - accuracy: 0.7722 - loss: 0.4840 - val_accuracy: 0.7853 - val_loss: 0.4760


In [17]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.7869 - loss: 0.4739
Loss:  0.4750058948993683
Accuracy:  0.7862943410873413


In [ ]:
#imbalanced data?
#data augmentation?